### Trial Graph NN

In [48]:
import pandas as pd

train_data = pd.read_pickle('Data/train_dataset_title_sim.pkl')
test_data = pd.read_pickle('Data/test_dataset_title_sim.pkl')
train_labels = train_data['label']
test_labels = test_data['label']

# Display the first few rows of the training and testing datasets
print("Training Data:")
display(train_data.head())
print("\nTesting Data:")
display(test_data.head())

Training Data:


,paper_a,paper_b,label,title,year,venue,index,citations,abstract,category_0,...,category_26_b,category_27_b,category_28_b,category_29_b,category_30_b,category_31_b,category_32_b,category_33_b,category_34_b,title_similarity
0,223465,481995,0,Generation and transformation of interface tra...,1993,1,223465,None,None,0,...,0,0,0,0,0,0,0,0,0,0.000000
1,347910,291414,1,An integrated modelling framework to support m...,2008,1,347910,110359;291414;593858;604335;433150,This paper proposes an integrated modelling fr...,0,...,0,0,0,0,0,0,0,0,0,0.578392
2,141221,422833,1,Bandit-based optimization on graphs with appli...,2009,1,141221,104308;422833,The problem of choosing fast implementations f...,0,...,0,0,0,0,0,0,0,0,1,0.000000
3,282962,229639,1,Completion time multiple branch prediction for...,2000,1,282962,83477;85142;229639;289424;591931;591451;449247...,The need for multiple branch prediction is inh...,0,...,0,0,0,0,0,0,0,0,1,0.426018
4,598712,468674,1,Sample sizes for sigmoidal neural networks,1995,1,598712,228084;457547;468674;486038;587976;600775,None,0,...,0,0,0,0,0,0,0,0,0,0.013893



Testing Data:


,paper_a,paper_b,label,title,year,venue,index,citations,abstract,category_0,...,category_26_b,category_27_b,category_28_b,category_29_b,category_30_b,category_31_b,category_32_b,category_33_b,category_34_b,title_similarity
0,119995,358509,0,An object-oriented framework for modular resou...,1996,1,119995,None,The authors present a flexible object-oriented...,0,...,0,0,0,0,0,0,0,0,1,0.000000
1,391615,416623,1,Ispy: detecting ip prefix hijacking on my own,2008,1,391615,21144;29093;49036;124539;338587;353602;416623;...,IP prefix hijacking remains a major threat to ...,0,...,0,0,0,0,0,0,0,0,1,0.518576
2,123767,452007,1,Tellegen's principle into practice,2003,1,123767,452007;460955;192531;287858;570231;286329;4877...,"The transposition principle, also called Telle...",0,...,0,0,0,0,0,0,0,0,0,0.000000
3,311521,376606,1,Efficient region-based image retrieval,2003,1,311521,174368;255539;291575;370858;366139;369052;3835...,Region-based image retrieval(RBIR) was recentl...,0,...,0,0,0,0,0,0,0,0,0,0.000000
4,341400,371412,1,BibFinder/StatMiner: effectively mining and us...,2003,1,341400,371412;623744,Recent work in data integration has shown the ...,0,...,0,0,0,0,0,0,0,0,0,0.318530


In [49]:
import pandas as pd

# Assuming train_data and test_data are already defined DataFrames

# Function to create a balanced dataset
def create_balanced_dataset(data, n_per_class=500):
    # Separate the data into two groups based on the 'label'
    group_0 = data[data['label'] == 0]
    group_1 = data[data['label'] == 1]

    # Randomly sample n_per_class rows from each group
    sampled_group_0 = group_0.sample(n=n_per_class, random_state=55)  # Set random_state for reproducibility
    sampled_group_1 = group_1.sample(n=n_per_class, random_state=55)

    # Combine the sampled groups to create a balanced dataset
    balanced_data = pd.concat([sampled_group_0, sampled_group_1])
    
    return balanced_data

# Create balanced training dataset
balanced_train_data = create_balanced_dataset(train_data, n_per_class=500)

# Drop the specified columns from the training dataset
X_train = balanced_train_data.drop(columns=['title', 'abstract', 'citations', 'index', 'label'])
y_train = balanced_train_data['label']

# Create balanced testing dataset
balanced_test_data = create_balanced_dataset(test_data, n_per_class=500)

# Drop the specified columns from the testing dataset
X_test = balanced_test_data.drop(columns=['title', 'abstract', 'citations', 'index', 'label'])
y_test = balanced_test_data['label']

# Display the first few rows of the training and testing datasets
print("Training Data:")
display(X_train)

print("Testing Data:")
display(X_test)

# Optionally, check the balance of the labels in the training and testing sets
print("Label distribution in training data:")
print(y_train.value_counts())

print("Label distribution in testing data:")
print(y_test.value_counts())

Training Data:


,paper_a,paper_b,year,venue,category_0,category_1,category_2,category_3,category_4,category_5,...,category_26_b,category_27_b,category_28_b,category_29_b,category_30_b,category_31_b,category_32_b,category_33_b,category_34_b,title_similarity
139911,572478,518617,2002,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
165378,500988,407451,2009,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
16168,556680,607463,2000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
56426,448817,189920,1995,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.018672
163921,382278,440110,1997,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75522,299897,541129,1998,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.000000
80764,441640,164950,2003,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.000000
199012,441842,173093,1990,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.230021
182090,219361,184506,1994,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0.248013


Testing Data:


,paper_a,paper_b,year,venue,category_0,category_1,category_2,category_3,category_4,category_5,...,category_26_b,category_27_b,category_28_b,category_29_b,category_30_b,category_31_b,category_32_b,category_33_b,category_34_b,title_similarity
43951,520679,252816,1992,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.025307
39639,119748,244891,2003,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.012532
40630,578083,165339,2005,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0.000000
25198,3579,628759,1998,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.000000
49256,406627,164979,2008,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42670,392499,91363,2006,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.253257
8976,463002,150387,1988,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.561113
21620,45304,296283,2008,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.120208
46461,287971,163009,1994,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.175088


Label distribution in training data:
label
0    500
1    500
Name: count, dtype: int64
Label distribution in testing data:
label
0    500
1    500
Name: count, dtype: int64


In [50]:
import networkx as nx

def create_graph(pairs_df):
    G = nx.DiGraph()  # Directed graph
    for _, row in pairs_df.iterrows():
        # Add nodes for paper_a and paper_b
        G.add_node(row['paper_a'], features=row[['year'] + ['venue'] + [f'category_{i}' for i in range(35)]].to_dict())
        G.add_node(row['paper_b'], features=row[['year_b'] + ['venue_b'] + [f'category_{i}_b' for i in range(35)]].to_dict())
        
        # Add edge from paper_a to paper_b
        G.add_edge(row['paper_a'], row['paper_b'], label=row['label'])
    
    return G

# Create training and testing graphs
# train_graph = create_graph(train_data)
# test_graph = create_graph(test_data)

In [51]:
#len(train_graph.nodes), len(train_graph.edges)

In [103]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler

# Step 1: Load and Prepare Data
train_data = pd.read_pickle('Data/train_dataset_title_sim.pkl')
test_data = pd.read_pickle('Data/test_dataset_title_sim.pkl')

X_train_raw = train_data.drop(columns=['title', 'title_b', 'abstract', 'abstract_b', 'citations', 'citations_b', 'index', 'index_b', 'label'])
y_train = train_data['label']
X_test_raw = test_data.drop(columns=['title', 'title_b', 'abstract', 'abstract_b', 'citations', 'citations_b', 'index', 'index_b', 'label'])
y_test = test_data['label']

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)


In [104]:
X_train_raw.columns

Index(['paper_a', 'paper_b', 'year', 'venue', 'category_0', 'category_1',
       'category_2', 'category_3', 'category_4', 'category_5', 'category_6',
       'category_7', 'category_8', 'category_9', 'category_10', 'category_11',
       'category_12', 'category_13', 'category_14', 'category_15',
       'category_16', 'category_17', 'category_18', 'category_19',
       'category_20', 'category_21', 'category_22', 'category_23',
       'category_24', 'category_25', 'category_26', 'category_27',
       'category_28', 'category_29', 'category_30', 'category_31',
       'category_32', 'category_33', 'category_34', 'year_b', 'venue_b',
       'category_0_b', 'category_1_b', 'category_2_b', 'category_3_b',
       'category_4_b', 'category_5_b', 'category_6_b', 'category_7_b',
       'category_8_b', 'category_9_b', 'category_10_b', 'category_11_b',
       'category_12_b', 'category_13_b', 'category_14_b', 'category_15_b',
       'category_16_b', 'category_17_b', 'category_18_b', 'category_19_b

In [105]:
def create_edge_index(data):
    # Create a mapping from unique paper IDs to numeric IDs
    paper_ids = pd.concat([data['paper_a'], data['paper_b']]).unique()
    paper_id_map = {paper: idx for idx, paper in enumerate(paper_ids)}

    # Replace paper_a and paper_b with their corresponding numeric IDs
    edge_index = []
    for i, row in data.iterrows():
        if row['label'] == 1:  # Add an edge if there's a citation
            edge_index.append([paper_id_map[row['paper_a']], paper_id_map[row['paper_b']]])  # Directed edge
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()


edge_index_train = create_edge_index(train_data)
edge_index_test = create_edge_index(test_data)

# Step 3: Prepare PyTorch Geometric Data Objects
data_train = Data(x=torch.tensor(X_train, dtype=torch.float),
                  edge_index=edge_index_train,
                  y=torch.tensor(y_train.values, dtype=torch.float))  # Binary target

data_test = Data(x=torch.tensor(X_test, dtype=torch.float),
                 edge_index=edge_index_test,
                 y=torch.tensor(y_test.values, dtype=torch.float))


In [106]:
data_train, data_test

(Data(x=[200000, 77], edge_index=[2, 100000], y=[200000]),
 Data(x=[49936, 77], edge_index=[2, 24936], y=[49936]))

In [107]:
# Step 4: Define GCN Model
class GCN(torch.nn.Module):
    def __init__(self, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, 1)  # Binary output

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x).squeeze()  # Binary probability output

# Step 5: Train GCN Model
def train_model(model, data, epochs=1000, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.binary_cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
    return model

# Initialize and train GCN
#gcn = GCN(num_node_features=X_train.shape[1])
#gcn = train_model(gcn, data_train)



In [108]:
# Step 6: Evaluate GCN Model
def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data)
        predictions = (out > 0.5).float()  # Convert probabilities to binary predictions
        accuracy = (predictions == data.y).sum().item() / len(data.y)
        print(f"Accuracy: {accuracy:.4f}")
        return predictions

print("\nEvaluating GCN:")
#gcn_predictions = evaluate_model(gcn, data_test)


Evaluating GCN:


## improved model

In [109]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

class ImprovedGCN(nn.Module):
    def __init__(self, num_node_features, num_classes, hidden_units=64, dropout_rate=0.5):
        super(ImprovedGCN, self).__init__()
        
        self.conv1 = GCNConv(num_node_features, hidden_units)
        self.conv2 = GCNConv(hidden_units, hidden_units)
        self.conv3 = GCNConv(hidden_units, num_classes)  # Third layer
        
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First Convolutional Layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)  # Apply dropout
        
        # Second Convolutional Layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Third Convolutional Layer
        x = self.conv3(x, edge_index)
        
        return torch.sigmoid(x)  # Use sigmoid for binary classification

In [110]:
def train_model(model, data, epochs=100, lr=0.01, patience=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    best_loss = float('inf')
    counter = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data).squeeze()  # Squeeze to match the shape of the labels
        loss = F.binary_cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        
        # Early stopping check
        if loss.item() < best_loss:
            best_loss = loss.item()
            counter = 0
        else:
            counter += 1
            if counter > patience:
                print(f"Early stopping at epoch {epoch}")
                break
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
    
    return model


In [111]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def evaluate_model(model, data):
    model.eval()
    with torch.no_grad():
        out = model(data).squeeze()  # Squeeze to match the labels shape
        pred = (out > 0.5).float()  # Convert to binary predictions (0 or 1)
        
        # Compute accuracy
        accuracy = (pred == data.y).sum().item() / len(data.y)
        print(f"Accuracy: {accuracy:.4f}")
        
        return out, pred

In [112]:
# Initialize and train the Improved GCN
#gcn = ImprovedGCN(num_node_features=X_train.shape[1], num_classes=1)
#gcn = train_model(gcn, data_train, epochs=1000, lr=0.005, patience=15)

Epoch 0, Loss: 0.8284
Epoch 10, Loss: 0.6670
Epoch 20, Loss: 0.6221
Epoch 30, Loss: 0.5923
Epoch 40, Loss: 0.5761
Epoch 50, Loss: 0.5649
Epoch 60, Loss: 0.5564
Epoch 70, Loss: 0.5489
Epoch 80, Loss: 0.5429
Epoch 90, Loss: 0.5393
Epoch 100, Loss: 0.5355
Epoch 110, Loss: 0.5328
Epoch 120, Loss: 0.5300
Epoch 130, Loss: 0.5275
Epoch 140, Loss: 0.5253
Epoch 150, Loss: 0.5236
Epoch 160, Loss: 0.5233
Epoch 170, Loss: 0.5224
Epoch 180, Loss: 0.5209
Epoch 190, Loss: 0.5208
Epoch 200, Loss: 0.5194
Epoch 210, Loss: 0.5192
Epoch 220, Loss: 0.5182
Epoch 230, Loss: 0.5178
Epoch 240, Loss: 0.5178
Epoch 250, Loss: 0.5167
Epoch 260, Loss: 0.5168
Epoch 270, Loss: 0.5158
Epoch 280, Loss: 0.5156
Epoch 290, Loss: 0.5158
Epoch 300, Loss: 0.5158
Epoch 310, Loss: 0.5156
Epoch 320, Loss: 0.5157
Early stopping at epoch 328


In [113]:
print("\nEvaluating GCN:")
#gcn_predictions = evaluate_model(gcn, data_test)


Evaluating GCN:


Not really improved

## Trying new method

In [144]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Step 1: Load and Prepare Data
train_data = pd.read_pickle('Data/train_dataset_title_sim.pkl')
test_data = pd.read_pickle('Data/test_dataset_title_sim.pkl')

# Extracting the label column
y_train = train_data['label']
y_test = test_data['label']


In [145]:
train_data.columns, test_data.columns

(Index(['paper_a', 'paper_b', 'label', 'title', 'year', 'venue', 'index',
        'citations', 'abstract', 'category_0', 'category_1', 'category_2',
        'category_3', 'category_4', 'category_5', 'category_6', 'category_7',
        'category_8', 'category_9', 'category_10', 'category_11', 'category_12',
        'category_13', 'category_14', 'category_15', 'category_16',
        'category_17', 'category_18', 'category_19', 'category_20',
        'category_21', 'category_22', 'category_23', 'category_24',
        'category_25', 'category_26', 'category_27', 'category_28',
        'category_29', 'category_30', 'category_31', 'category_32',
        'category_33', 'category_34', 'title_b', 'year_b', 'venue_b', 'index_b',
        'citations_b', 'abstract_b', 'category_0_b', 'category_1_b',
        'category_2_b', 'category_3_b', 'category_4_b', 'category_5_b',
        'category_6_b', 'category_7_b', 'category_8_b', 'category_9_b',
        'category_10_b', 'category_11_b', 'category_12_b',

In [146]:

# Remove the columns that are not needed for features
def preprocess_data(df):
    features = []
    for col in df.columns:
        if col.startswith('category'):
            features.append(df[col].values)  # Category features
    
    # Add additional features (e.g., year, venue, title similarity)
    features.append(df['year'].values)
    features.append(df['venue'].values)
    features.append(df['year_b'].values)
    features.append(df['venue_b'].values)
    features.append(df['title_similarity'].values)
    
    # Convert the features to a 2D array (each row is a paper pair)
    features = np.column_stack(features)
    
    return features

# Step 2: Preprocess the raw train and test data
X_train = preprocess_data(train_data)
X_test = preprocess_data(test_data)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 3: Calculate number of unique nodes
num_nodes = len(pd.concat([train_data['paper_a'], train_data['paper_b']]).unique())

# Step 4: Create edge_index
def create_edge_index(data, num_nodes):
    # Create a mapping from unique paper IDs to numeric indices (0, 1, 2,...)
    paper_ids = pd.concat([data['paper_a'], data['paper_b']]).unique()
    paper_id_map = {paper: idx for idx, paper in enumerate(paper_ids)}

    # Replace paper_a and paper_b with their corresponding numeric IDs
    edge_index = []
    for i, row in data.iterrows():
        if row['label'] == 1:  # Add an edge if there's a citation
            paper_a_idx = paper_id_map[row['paper_a']]
            paper_b_idx = paper_id_map[row['paper_b']]
            
            # Ensure the indices are valid
            if paper_a_idx < num_nodes and paper_b_idx < num_nodes:
                edge_index.append([paper_a_idx, paper_b_idx])  # Directed edge
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

edge_index_train = create_edge_index(train_data, num_nodes)
edge_index_test = create_edge_index(test_data, num_nodes)

# Step 5: Define a custom Dataset class for PyTorch Geometric
class GraphDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, edge_index):
        self.X = X
        self.y = y
        self.edge_index = edge_index

    def __len__(self):
        return len(self.X)  # Return the number of data samples

    def __getitem__(self, idx):
        # For each sample, return a Data object (used by DataLoader)
        data = Data(x=self.X[idx], edge_index=self.edge_index, y=self.y[idx])
        return data

# Convert the features to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float)

# Step 6: Create Dataset and DataLoader objects
train_dataset = GraphDataset(X_train_tensor, y_train_tensor, edge_index_train)
test_dataset = GraphDataset(X_test_tensor, y_test_tensor, edge_index_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [149]:
def validate_edge_index(edge_index, num_nodes):
    # Check if any indices are out of range
    if edge_index.max() >= num_nodes or edge_index.min() < 0:
        print(f"Error: edge_index contains out-of-range indices.")
        print(f"Max index: {edge_index.max()}, Number of nodes: {num_nodes}")
        print(f"Min index: {edge_index.min()}")
        return False
    return True

# Validate edge_index after creation
if not validate_edge_index(edge_index_train, num_nodes):
    print("Invalid edge_index in training data!")
if not validate_edge_index(edge_index_test, num_nodes):
    print("Invalid edge_index in test data!")


In [147]:

# Step 6: Define the Improved Graph Neural Network model
class ImprovedGCN(nn.Module):
    def __init__(self, num_node_features, num_classes=1, hidden_units=64, dropout_rate=0.5):
        super(ImprovedGCN, self).__init__()
        
        self.conv1 = GCNConv(num_node_features, hidden_units)
        self.conv2 = GCNConv(hidden_units, hidden_units)
        self.conv3 = GCNConv(hidden_units, num_classes)  # Third layer
        
        self.dropout = nn.Dropout(dropout_rate)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # First Convolutional Layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)  # Apply dropout
        
        # Second Convolutional Layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Third Convolutional Layer
        x = self.conv3(x, edge_index)
        
        return torch.sigmoid(x)  # Use sigmoid for binary classification

def train_model(model, train_loader, epochs=50):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(epochs):
        model.train()
        for data in train_loader:
            # Debugging: Inspect the shapes
            print(f"Data.x shape: {data.x.shape}")  # Expected to be [num_nodes, num_features]
            print(f"Data.edge_index shape: {data.edge_index.shape}")  # Expected to be [2, num_edges]
            
            # Check for out-of-bounds indices in edge_index
            if data.edge_index.max() >= data.x.size(0):
                print(f"Warning: edge_index contains indices larger than the number of nodes: {data.edge_index.max().item()} >= {data.x.size(0)}")

            # Ensure data.x is of shape (num_nodes, num_features)
            if len(data.x.shape) == 1:
                data.x = data.x.view(-1, 1)  # Reshape to (num_nodes, 1) if it's currently (num_features,)
                print(f"Reshaped Data.x shape: {data.x.shape}")

            optimizer.zero_grad()
            out = model(data).squeeze()  # Squeeze to match the shape of the labels
            loss = F.binary_cross_entropy(out, data.y)
            loss.backward()
            optimizer.step()
            
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Step 8: Evaluation function (with ROC Curve)
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for data in test_loader:
            out = model(data).squeeze()  # Squeeze to match the labels shape
            pred = (out > 0.5).float()  # Convert to binary predictions (0 or 1)
            all_preds.append(out)
            all_labels.append(data.y)
        
    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()
    
    # Compute accuracy
    accuracy = (all_preds.round() == all_labels).sum() / len(all_labels)
    print(f"Accuracy: {accuracy:.4f}")
    
    # ROC Curve and AUC
    fpr, tpr, _ = roc_curve(all_labels, all_preds)
    roc_auc = auc(fpr, tpr)
    print(f"AUC: {roc_auc:.4f}")
    
    # Plot ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Step 9: Initialize the Model, Optimizer, and Loss Function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ImprovedGCN(num_node_features=X_train.shape[1]).to(device)

In [148]:
# Step 10: Train the model
train_model(model, train_loader, epochs=50)

# Step 11: Evaluate on test set
evaluate_model(model, test_loader)


Data.x shape: torch.Size([9600])
Data.edge_index shape: torch.Size([2, 12800000])
Reshaped Data.x shape: torch.Size([9600, 1])


RuntimeError: index 68927 is out of bounds for dimension 0 with size 9600